In [1]:
import collections
import nltk.classify.util, nltk.metrics
from nltk.classify import NaiveBayesClassifier, MaxentClassifier, SklearnClassifier
import csv
from sklearn import cross_validation
from sklearn.svm import LinearSVC, SVC
import random
from nltk.corpus import stopwords
import itertools
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures

c:\program files\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from __future__ import division #To avoid integer division
from operator import itemgetter
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv("data_set.csv", low_memory=False)
data = data[['reviews.rating', 'reviews.text']]
train, test = train_test_split(data, test_size=0.2)

In [4]:
pos_rev = []
neg_rev = []
neut_rev = []

train['reviews.rating'] = pd.to_numeric(train['reviews.rating'], errors='coerce')
train = train.dropna(subset=['reviews.rating'])
train['reviews.rating'] = train['reviews.rating'].astype(int)

# a = 0
for col, row in train.iterrows():
    if int(row["reviews.rating"]) > 3:
        pos_rev.append(row["reviews.text"])
    elif int(row["reviews.rating"]) < 3:
        neg_rev.append(row["reviews.text"])
    else:
        neut_rev.append(row["reviews.text"])

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [5]:
pos_rev

["Alexa has become a member of the family. She wakes us up in the morning, helps with math homework, plays music, and keeps track of our shopping lists! I can't imagine not having this device and the Dots in every room!",
 'The set up was quick and simple. The voice commands work well.',
 'I was going to get a Samsung tablet but for doing everything this is better',
 "For starters i had the Amazon stick for a while & got tired of that due to all the buffering issues, but what can you expect out of that little stick? so when this went on sale, i decided to upgrade & i'am so happy i did, it works great & has reduced the buffering to a minimum, but like anything online depends on the site your on. Love the fact you can speak into the remote and pull up the site you want.",
 'Every day Alexa grows smarter and my music pick just grows with any mood I want to hear . I love that more apps are coming for this product',
 'Great product like just as much as the fire stick great',
 'Love the fire

In [13]:
def word_split(data):    
    data_new = []
    for word in data:
        try:
            word_filter = [i.lower() for i in word.split()]
            data_new.append(word_filter)
        except:
            print (' ')
    return data_new
 
def word_split_sentiment(data):
    data_new = []
    for (word, sentiment) in data:
        try:
            word_filter = [i.lower() for i in word.split()]
            data_new.append((word_filter, sentiment))
        except:
            print (' ')
    return data_new
    
def word_feats(words):    
    return dict([(word, True) for word in words])

In [46]:
def evaluate_classifier(featx):
    
    negfeats = [(featx(f), 'neg') for f in word_split(neg_rev)]
    posfeats = [(featx(f), 'pos') for f in word_split(pos_rev)]
    
#     print (posfeats)
        
    negcutoff = int(len(negfeats)*3/4)
    poscutoff = int(len(posfeats)*3/4)
    
    print (len(posfeats))
    print (len(negfeats))
 
    trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
    testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
    
    # using 3 classifiers
    classifier_list = ['nb', 'maxent', 'svm']     
        
    for cl in classifier_list:
        if cl == 'maxent':
            classifierName = 'Maximum Entropy'
            classifier = MaxentClassifier.train(trainfeats, 'GIS', trace=0, encoding=None, labels=None, gaussian_prior_sigma=0, max_iter = 1)
        elif cl == 'svm':
            classifierName = 'SVM'
            classifier = SklearnClassifier(LinearSVC(), sparse=False)
            classifier.train(trainfeats)
        else:
            classifierName = 'Naive Bayes'
            classifier = NaiveBayesClassifier.train(trainfeats)
            
        refsets = collections.defaultdict(set)
        testsets = collections.defaultdict(set)
 
        for i, (feats, label) in enumerate(testfeats):
                refsets[label].add(i)
                observed = classifier.classify(feats)
                testsets[observed].add(i)
 
        accuracy = nltk.classify.util.accuracy(classifier, testfeats)
#         pos_precision = nltk.metrics.precision(refsets['pos'], testsets['pos'])
#         pos_recall = nltk.metrics.recall(refsets['pos'], testsets['pos'])
#         pos_fmeasure = nltk.metrics.f_measure(refsets['pos'], testsets['pos'])
#         neg_precision = nltk.metrics.precision(refsets['neg'], testsets['neg'])
#         neg_recall = nltk.metrics.recall(refsets['neg'], testsets['neg'])
#         neg_fmeasure =  nltk.metrics.f_measure(refsets['neg'], testsets['neg'])
        
        print (' ')
        print ('---------------------------------------')
        print ('SINGLE FOLD RESULT ' + '(' + classifierName + ')')
        print ('---------------------------------------')
        print ('accuracy:', accuracy)
#         print ('precision', (pos_precision + neg_precision) / 2)
#         print ('recall', (pos_recall + neg_recall) / 2)
#         print ('f-measure', (pos_fmeasure + neg_fmeasure) / 2)
                
        #classifier.show_most_informative_features()
        
    trainfeats = negfeats + posfeats    
    
    # SHUFFLE TRAIN SET
    # As in cross validation, the test chunk might have only negative or only positive data    
    random.shuffle(trainfeats)    
    n = 5 # 5-fold cross-validation    
    
    for cl in classifier_list:
        
        subset_size = int(len(trainfeats) / n)
        accuracy = []
        pos_precision = []
        pos_recall = []
        neg_precision = []
        neg_recall = []
        pos_fmeasure = []
        neg_fmeasure = []
        cv_count = 1
        for i in range(n):        
            testing_this_round = trainfeats[i*subset_size:][:subset_size]
            training_this_round = trainfeats[:i*subset_size] + trainfeats[(i+1)*subset_size:]
            
            if cl == 'maxent':
                classifierName = 'Maximum Entropy'
                classifier = MaxentClassifier.train(training_this_round, 'GIS', trace=0, encoding=None, labels=None, gaussian_prior_sigma=0, max_iter = 1)
            elif cl == 'svm':
                classifierName = 'SVM'
                classifier = SklearnClassifier(LinearSVC(), sparse=False)
                classifier.train(training_this_round)
            else:
                classifierName = 'Naive Bayes'
                classifier = NaiveBayesClassifier.train(training_this_round)
                    
            refsets = collections.defaultdict(set)
            testsets = collections.defaultdict(set)
            for i, (feats, label) in enumerate(testing_this_round):
                refsets[label].add(i)
                observed = classifier.classify(feats)
                testsets[observed].add(i)
            
            cv_accuracy = nltk.classify.util.accuracy(classifier, testing_this_round)
#             cv_pos_precision = nltk.metrics.precision(refsets['pos'], testsets['pos'])
#             cv_pos_recall = nltk.metrics.recall(refsets['pos'], testsets['pos'])
#             cv_pos_fmeasure = nltk.metrics.f_measure(refsets['pos'], testsets['pos'])
#             cv_neg_precision = nltk.metrics.precision(refsets['neg'], testsets['neg'])
#             cv_neg_recall = nltk.metrics.recall(refsets['neg'], testsets['neg'])
#             cv_neg_fmeasure =  nltk.metrics.f_measure(refsets['neg'], testsets['neg'])
                    
            accuracy.append(cv_accuracy)
#             pos_precision.append(cv_pos_precision)
#             pos_recall.append(cv_pos_recall)
#             neg_precision.append(cv_neg_precision)
#             neg_recall.append(cv_neg_recall)
#             pos_fmeasure.append(cv_pos_fmeasure)
#             neg_fmeasure.append(cv_neg_fmeasure)
            
            cv_count += 1
                
        print ('---------------------------------------')
        print ('N-FOLD CROSS VALIDATION RESULT ' + '(' + classifierName + ')')
        print ('---------------------------------------')
        print ('accuracy:', sum(accuracy) / n)
#         print ('precision', (sum(pos_precision)/n + sum(neg_precision)/n) / 2)
#         print ('recall', (sum(pos_recall)/n + sum(neg_recall)/n) / 2)
#         print ('f-measure', (sum(pos_fmeasure)/n + sum(neg_fmeasure)/n) / 2)
        print ('')
    
    

In [47]:
evaluate_classifier(word_feats)

 
25859
656
---------------------------------------
N-FOLD CROSS VALIDATION RESULT (Naive Bayes)
---------------------------------------
accuracy: 0.5163869507825759

---------------------------------------
N-FOLD CROSS VALIDATION RESULT (Maximum Entropy)
---------------------------------------
accuracy: 0.9752592871959269

---------------------------------------
N-FOLD CROSS VALIDATION RESULT (SVM)
---------------------------------------
accuracy: 0.9744295681689611

